# A sandbox to play in

In [ ]:
import sys, os
import json
from time import time
import glob

import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.io
import cv2
from PIL import Image

from torch_geometric.data import Data

# My libraries. Ugly hack to import from sister directory



sandbox_num = "0"
os.environ['CUDA_VISIBLE_DEVICES'] = sandbox_num
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # for debugging GPU stuff

from importlib import reload

In [ ]:
datasets_base_dir = '/data/'
def torch_to_numpy(torch_tensor, is_standardized_image = False):
    """ Converts torch tensor (NCHW) to numpy tensor (NHWC) for plotting
    
        If it's an rgb image, it puts it back in [0,255] range (and undoes ImageNet standardization)
    """
    np_tensor = torch_tensor.cpu().clone().detach().numpy()
    if np_tensor.ndim == 4: # NCHW
        np_tensor = np_tensor.transpose(0,2,3,1)
    if is_standardized_image:
        _mean=[0.485, 0.456, 0.406]; _std=[0.229, 0.224, 0.225]
        for i in range(3):
            np_tensor[...,i] *= _std[i]
            np_tensor[...,i] += _mean[i]
        np_tensor *= 255
            
    return np_tensor

# Introduction Tutorial: torch_geometric.Data

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

In [ ]:
dir(data)[34:]

In [ ]:
data.is_directed()

In [ ]:
print(data.keys)

In [ ]:
print(data['x'])

In [ ]:
for key, item in data:
    print("{} found in data".format(key))

In [ ]:
'edge_attr' in data

In [ ]:
data.num_nodes

In [ ]:
data.num_edges

In [ ]:
data.num_node_features

In [ ]:
data.contains_isolated_nodes()

In [ ]:
data.contains_self_loops()

In [ ]:
data.is_directed()

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

# Checkout torch_geometric.dataset

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

In [ ]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [ ]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)
data = dataset[0]
data

In [ ]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)

# GCN Tutorial

In [ ]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)
data = dataset[0]
data

In [ ]:
print(data.train_mask.sum().item())
print(data.val_mask.sum().item())
print(data.test_mask.sum().item())

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))